# Scraping von Wikipedia

Zunächst müssen die notwendigen Bibliotheken und Module eingebunden werden.

In [1]:
from urllib.request import urlopen

Daraufhin bauen wir uns zunächst eine funktionierende URL zusammen, die auf unsere Startseite verweist. Wir trennen die `base_url` und den Suchbegriff `term`, damit wir später auf die `base_url` wieder zugreifen können.

In [5]:
base_url = 'https://de.wikipedia.org/wiki/'
term = '26._November'
search = base_url + term

In [6]:
search

'https://de.wikipedia.org/wiki/26._November'

Anschließend rufen wir die Seite namens `search` programmatisch auf und speichern die Antwort des Servers unter der Variable `html` ab.

In [8]:
html = urlopen(search)

Um mit der Serverantwort weiterarbeiten zu können, benötigen wir eine weitere Bibliothek namens `BeautifulSoup`. Mit dieser lassen sich die Serverantworten parsen, das heißt verarbeiten und auswerten.

In [12]:
from bs4 import BeautifulSoup

Wir erstellen ein BeautifulSoup-Objekt mit unserer Serverantwort und einem HTML-Parser.

In [13]:
soup = BeautifulSoup(html, 'html.parser')

Jetzt machen wir uns auf die Suche nach dem Abschnitt, der in Form einer ungeordneten Aufzählung die Links zu den einzelnen Personeneinträgen beinhaltet, die vor dem 19. Jahrhundert am 26. November geboren worden sind.

Wir greifen dafür auf unser `soup`-Objekt zu und wenden auf ihm die Methode `find()` an. Der Methode übergeben wir als Argument den Tag, den wir suchen wollen. In unserem Fall `"span"`. Unser `span` hat aber eine bestimmte `id` namens `"Vor_dem_19._Jahrhundert"`. Von dort springen wir zunächst zum Eltern-Tag und dann weitern zum nächsten Geschwister-Tag. Dabei handelt es sich um unsere ungeordnete Liste `<ul>`. Aus dieser Liste greifen wir mit `find_all("li")` auf alle Listen-Elemente zu. Diese speichern wir in der Variable `section` ab.

In [21]:
section = soup.find("span", {"id": "Vor_dem_19._Jahrhundert"}).find_parent().find_next_sibling().find_all("li")

In [24]:
section

[<li><span style="visibility:hidden;">0</span><a href="/wiki/117" title="117">117</a>: <a href="/wiki/Aelius_Aristides" title="Aelius Aristides">Aelius Aristides</a>, griechischer Rhetor und Schriftsteller</li>,
 <li><span style="visibility:hidden;">0</span><a href="/wiki/907" title="907">907</a>: <a href="/wiki/Rosendo_de_Celanova" title="Rosendo de Celanova">Rosendo de Celanova</a>, Bischof von Mondoñedo</li>,
 <li><a href="/wiki/1288" title="1288">1288</a>: <a href="/wiki/Go-Daigo" title="Go-Daigo">Go-Daigo</a>, 96. Kaiser von Japan</li>,
 <li><a href="/wiki/1350" title="1350">1350</a>: <a href="/wiki/Ibn_al-Dschazar%C4%AB" title="Ibn al-Dschazarī">Ibn al-Dschazarī</a>, islamischer Rechts- und Korangelehrter</li>,
 <li><a href="/wiki/1462" title="1462">1462</a>: <a href="/wiki/Alexander_(Pfalz-Zweibr%C3%BCcken)" title="Alexander (Pfalz-Zweibrücken)">Alexander von Pfalz-Zweibrücken</a>, deutscher Adliger, Herzog von Zweibrücken und Veldenz</li>,
 <li><a href="/wiki/1588" title="1588"

Bei `section` handelt es sich bei genauerem Hinsehen um eine Liste.

Als nächstes erstellen wir eine leere Liste namens `links`, in die wir die eigentlichen Links aus den List-Elementen ablegen.

Um an den Link zu den Personenseiten zu gelangen, müssen wir zunächst mit `find("a")` den ersten Link in dem Element finden. Anschließend suchen wir das nächste Geschwister-Element und greifen auf das Attribut `"href"` in eckigen Klammern zurück.

In [43]:
links = []
for list_item in section:
    links.append(list_item.find("a").find_next_sibling()["href"].lstrip("/wik"))

In [44]:
links

['Aelius_Aristides',
 'Rosendo_de_Celanova',
 'Go-Daigo',
 'Ibn_al-Dschazar%C4%AB',
 'Alexander_(Pfalz-Zweibr%C3%BCcken)',
 'Anton_von_Ditfurth',
 'Johann_Bach',
 'John_Harvard_(Theologe)',
 'William_Derham',
 'Georg_Albrecht_Hamberger',
 'Karl_Leopold_(Mecklenburg)',
 'William_Cowper_(Dichter)',
 'Georg_Cancrin',
 'Karl_Philipp_Fohr',
 'Anton_Martin_Slom%C5%A1ek']

In [52]:
from urllib.parse import unquote

In [53]:
for link in links:
    url = base_url + link
    print(url)
    treffer = urlopen(url)
    person = BeautifulSoup(treffer, 'html.parser')
    #print(person.get_text())
    with open(unquote(link) + '.txt', 'w') as f:
        f.write(person.get_text())

https://de.wikipedia.org/wiki/Aelius_Aristides
https://de.wikipedia.org/wiki/Rosendo_de_Celanova
https://de.wikipedia.org/wiki/Go-Daigo
https://de.wikipedia.org/wiki/Ibn_al-Dschazar%C4%AB
https://de.wikipedia.org/wiki/Alexander_(Pfalz-Zweibr%C3%BCcken)
https://de.wikipedia.org/wiki/Anton_von_Ditfurth
https://de.wikipedia.org/wiki/Johann_Bach
https://de.wikipedia.org/wiki/John_Harvard_(Theologe)
https://de.wikipedia.org/wiki/William_Derham
https://de.wikipedia.org/wiki/Georg_Albrecht_Hamberger
https://de.wikipedia.org/wiki/Karl_Leopold_(Mecklenburg)
https://de.wikipedia.org/wiki/William_Cowper_(Dichter)
https://de.wikipedia.org/wiki/Georg_Cancrin
https://de.wikipedia.org/wiki/Karl_Philipp_Fohr
https://de.wikipedia.org/wiki/Anton_Martin_Slom%C5%A1ek
